# Preparing Data

우리가 하려는 건 결국 주식에 대한 정보와, 과거 주가를 넣으면 미래 주가가 나오는 모델을 만드는 것이 목표. <br>
그러면 시기가 끊기는 것은 중요하지 않으므로, NaN 값이 없는 데이터셋을 사용

## Data 불러오기 (colab)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset-evalPRPR-SPX-nonan (2).csv')

## Data 불러오기 (VS code)

In [ ]:
import pandas as pd
df = pd.read_csv("./dataset-evalPRPR-SPX-nonan (2).csv")

## 기본 데이터 정보 확인

In [3]:
df_ = df.copy()

In [4]:
# 결측치 확인 - 없음
df.isna().describe()

,ticker,company,timestamp,financial stability and liquidity,strong management team,competitive advantage,market potential,growth prospects,diversification within the company,sustainable business model,...,low,close,volume,return,SPX,calculated_price,P,P_future,R,R_future
count,241298,241298,241298,241298,241298,241298,241298,241298,241298,241298,...,241298,241298,241298,241298,241298,241298,241298,241298,241298,241298
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,241298,241298,241298,241298,241298,241298,241298,241298,241298,241298,...,241298,241298,241298,241298,241298,241298,241298,241298,241298,241298


In [5]:
# list가 아닌 str로 저장되어 있음
type(df['R_future'][33])

str

In [6]:
df.columns

Index(['ticker', 'company', 'timestamp', 'financial stability and liquidity',
       'strong management team', 'competitive advantage', 'market potential',
       'growth prospects', 'diversification within the company',
       'sustainable business model', 'innovation and R&D',
       'corporate governance', 'strong brand recognition', 'open', 'high',
       'low', 'close', 'volume', 'return', 'SPX', 'calculated_price', 'P',
       'P_future', 'R', 'R_future'],
      dtype='object')

# Data Preprocessing

## 모든 timestamp 정보를 가진 기업만 추리기

In [7]:
# 우리가 가지고 있는 모든 timestamp(509개)의 정보를 가진 기업만 다룰 예정
# 지워야 하는 기업의 리스트
delete_ticker_list = []

# for문으로 509개의 정보를 가지지 않은 기업 delete_ticker_list에 저장
for t in df['ticker'].unique():
  if not len(df.loc[df['ticker'] == t, 'timestamp'].values) == 509:
      delete_ticker_list.append(t)

# delete_ticker_list에 들어있지 않은 기업들만 데려감
for ticker in delete_ticker_list:
    df = df[df['ticker'] != ticker]

df.columns

Index(['ticker', 'company', 'timestamp', 'financial stability and liquidity',
       'strong management team', 'competitive advantage', 'market potential',
       'growth prospects', 'diversification within the company',
       'sustainable business model', 'innovation and R&D',
       'corporate governance', 'strong brand recognition', 'open', 'high',
       'low', 'close', 'volume', 'return', 'SPX', 'calculated_price', 'P',
       'P_future', 'R', 'R_future'],
      dtype='object')

## 정규화

In [8]:
# 정규화 
## GPT점수는 100점만점 척도이므로 기업별로 다르지 않음 따라서 통합 정규화
## open, high, low, close은 주가관련인데, 이는 기업별로 다르므로 기업별로 정규화 필요
## SPX은 기업별로 다르지 않음 따라서 통합 정규화
## volume은 기업별로 다르지만 통합 정규화가 올바름
## return은 따로 정규화할 필요는 없을거 같음
## 정규화는 편의를 위해 MinMaxScaler만 사용
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
unique_tickers = df['ticker'].unique()

columns_lists = df.columns.to_list()
columns_list = []
for c in columns_lists:
    if c not in ['company', 'P', 'P_future', 'R', 'R_future']:
        columns_list.append(c)
print(columns_list)
total_list = columns_list[2:12] + [columns_list[-1]] + [columns_list[-3]]
print(total_list)
seperate_list = columns_list[12:16]
print(seperate_list)

# 통합 정규화 : spx, gpt, volume
for i in total_list:
    df[i] = scaler.fit_transform(df[i].values.reshape(-1, 1))

# 개별기업별 정규화 : open, high, low, close	
for i in unique_tickers:
    for j in seperate_list:
        mask = (df['ticker'] == i)
        df.loc[mask, j] = scaler.fit_transform(df.loc[mask, j].values.reshape(-1, 1))

df

['ticker', 'timestamp', 'financial stability and liquidity', 'strong management team', 'competitive advantage', 'market potential', 'growth prospects', 'diversification within the company', 'sustainable business model', 'innovation and R&D', 'corporate governance', 'strong brand recognition', 'open', 'high', 'low', 'close', 'volume', 'return', 'SPX', 'calculated_price']
['financial stability and liquidity', 'strong management team', 'competitive advantage', 'market potential', 'growth prospects', 'diversification within the company', 'sustainable business model', 'innovation and R&D', 'corporate governance', 'strong brand recognition', 'calculated_price', 'return']
['open', 'high', 'low', 'close']


,ticker,company,timestamp,financial stability and liquidity,strong management team,competitive advantage,market potential,growth prospects,diversification within the company,sustainable business model,...,low,close,volume,return,SPX,calculated_price,P,P_future,R,R_future
0,MMM,3M,2013-05-03,0.714286,0.714286,0.7,0.571429,0.411765,0.916667,0.818182,...,0.000000,0.002051,16064000.0,0.123858,1614.42,0.018047,"[102.66, 103.23, 103.54, 103.77, 105.71, 106.4...","[110.48, 111.39, 110.27, 110.27, 111.11, 111.0...","[0.010831, 0.0055523, 0.003003, 0.0022214, 0.0...","[0.0245757, 0.0082368, -0.0100548, 0.0, 0.0076..."
1,MMM,3M,2013-05-10,0.714286,0.714286,0.7,0.571429,0.411765,0.916667,0.818182,...,0.032070,0.019588,14544300.0,0.122070,1633.70,0.018494,"[103.23, 103.54, 103.77, 105.71, 106.4, 106.42...","[111.39, 110.27, 110.27, 111.11, 111.03, 109.5...","[0.0055523, 0.003003, 0.0022214, 0.0186952, 0....","[0.0082368, -0.0100548, 0.0, 0.0076177, -0.000..."
2,MMM,3M,2013-05-17,0.714286,0.714286,0.7,0.571429,0.411765,0.916667,0.818182,...,0.047964,0.025610,14184300.0,0.120035,1667.47,0.018647,"[103.54, 103.77, 105.71, 106.4, 106.42, 106.31...","[110.27, 110.27, 111.11, 111.03, 109.59, 109.3...","[0.003003, 0.0022214, 0.0186952, 0.0065273, 0....","[-0.0100548, 0.0, 0.0076177, -0.00072, -0.0129..."
3,MMM,3M,2013-05-24,0.714286,0.714286,0.7,0.571429,0.411765,0.916667,0.818182,...,0.046417,0.018199,13560600.0,0.117756,1649.60,0.018458,"[103.77, 105.71, 106.4, 106.42, 106.31, 105.78...","[110.27, 111.11, 111.03, 109.59, 109.35, 111.5...","[0.0022214, 0.0186952, 0.0065273, 0.000188, -0...","[0.0, 0.0076177, -0.00072, -0.0129695, -0.0021..."
4,MMM,3M,2013-05-31,0.714286,0.714286,0.7,0.571429,0.411765,0.916667,0.818182,...,0.051902,0.018199,10642500.0,0.119009,1630.74,0.018458,"[105.71, 106.4, 106.42, 106.31, 105.78, 107.68...","[111.11, 111.03, 109.59, 109.35, 111.54, 115.1...","[0.0186952, 0.0065273, 0.000188, -0.0010336, -...","[0.0076177, -0.00072, -0.0129695, -0.00219, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241293,ZTS,Zoetis,2022-12-30,0.857143,0.892857,0.8,0.714286,0.647059,0.666667,0.909091,...,0.537044,0.547929,4950165.0,0.119684,3839.50,0.024571,"[147.37, 145.4, 147.21, 153.28, 133.67, 148.55...","[147.64, 160.92, 163.81, 165.18, 167.8, 158.33...","[-0.0062041, -0.0133677, 0.0124484, 0.0412336,...","[0.0074377, 0.0899485, 0.0179592, 0.0083633, 0..."
241294,ZTS,Zoetis,2023-01-06,0.857143,0.892857,0.8,0.714286,0.647059,0.666667,0.909091,...,0.527507,0.552984,6958752.0,0.119935,3895.08,0.024755,"[145.4, 147.21, 153.28, 133.67, 148.55, 145.64...","[160.92, 163.81, 165.18, 167.8, 158.33, 172.03...","[-0.0133677, 0.0124484, 0.0412336, -0.1279358,...","[0.0899485, 0.0179592, 0.0083633, 0.0158615, -..."
241295,ZTS,Zoetis,2023-01-13,0.857143,0.892857,0.8,0.714286,0.647059,0.666667,0.909091,...,0.553002,0.614571,12618342.0,0.130215,3999.09,0.026992,"[147.21, 153.28, 133.67, 148.55, 145.64, 150.1...","[163.81, 165.18, 167.8, 158.33, 172.03, 166.33...","[0.0124484, 0.0412336, -0.1279358, 0.1113189, ...","[0.0179592, 0.0083633, 0.0158615, -0.0564362, ..."
241296,ZTS,Zoetis,2023-01-20,0.857143,0.892857,0.8,0.714286,0.647059,0.666667,0.909091,...,0.607481,0.627974,13851912.0,0.121246,3972.61,0.027479,"[153.28, 133.67, 148.55, 145.64, 150.1, 157.42...","[165.18, 167.8, 158.33, 172.03, 166.33, 170.56...","[0.0412336, -0.1279358, 0.1113189, -0.0195894,...","[0.0083633, 0.0158615, -0.0564362, 0.0865281, ..."


## sequential 정보 처리

### 'timestamp' => 'year', 'month', 'day'

In [9]:
# 'date' 열을 datetime 타입으로 변환
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 'year', 'month', 'day' 열 추출
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

df.columns

Index(['ticker', 'company', 'timestamp', 'financial stability and liquidity',
       'strong management team', 'competitive advantage', 'market potential',
       'growth prospects', 'diversification within the company',
       'sustainable business model', 'innovation and R&D',
       'corporate governance', 'strong brand recognition', 'open', 'high',
       'low', 'close', 'volume', 'return', 'SPX', 'calculated_price', 'P',
       'P_future', 'R', 'R_future', 'year', 'month', 'day'],
      dtype='object')

### P, R to each columns

In [10]:
# P, P_future, R, R_future는 string을 value로 가지므로, 우리의 원 목적에 맞게 list로 형변환 진행

import ast

df['P'] = df['P'].apply(ast.literal_eval)
df['P_future'] = df['P_future'].apply(ast.literal_eval)
df['R'] = df['R'].apply(ast.literal_eval)
df['R_future'] = df['R_future'].apply(ast.literal_eval)

In [11]:
# sliding window의 효과를 위해 column명 작업 (새롭게 추가하고 싶은 column명을 리스트로 만들기)
# log('P') 준비도 한 번에
l = 13

lst = [i for i in range(-l+1, l+1)]
Pcol = []
Rcol= []

for j in ['logP', 'R']:
  for i in range(2*l):
    if lst[i] < 0:
      col_name = j + "_t" + str(lst[i])
      if j == 'logP': Pcol.append(col_name)
      else: Rcol.append(col_name)
    elif lst[i] == 0:
      col_name = j + "_t"
      if j == 'logP': Pcol.append(col_name)
      else: Rcol.append(col_name)
    else:
      col_name = j + "_t+" + str(lst[i])
      if j == 'logP': Pcol.append(col_name)
      else: Rcol.append(col_name)

print(Pcol)
print(Rcol)

['logP_t-12', 'logP_t-11', 'logP_t-10', 'logP_t-9', 'logP_t-8', 'logP_t-7', 'logP_t-6', 'logP_t-5', 'logP_t-4', 'logP_t-3', 'logP_t-2', 'logP_t-1', 'logP_t', 'logP_t+1', 'logP_t+2', 'logP_t+3', 'logP_t+4', 'logP_t+5', 'logP_t+6', 'logP_t+7', 'logP_t+8', 'logP_t+9', 'logP_t+10', 'logP_t+11', 'logP_t+12', 'logP_t+13']
['R_t-12', 'R_t-11', 'R_t-10', 'R_t-9', 'R_t-8', 'R_t-7', 'R_t-6', 'R_t-5', 'R_t-4', 'R_t-3', 'R_t-2', 'R_t-1', 'R_t', 'R_t+1', 'R_t+2', 'R_t+3', 'R_t+4', 'R_t+5', 'R_t+6', 'R_t+7', 'R_t+8', 'R_t+9', 'R_t+10', 'R_t+11', 'R_t+12', 'R_t+13']


In [12]:
# P, P_future, R, R_future 리스트에 있던 값을 각각 분리해서 기존 데이터 프레임에 추가
df[Pcol[:l]] = pd.DataFrame(df['P'].tolist(), index=df.index)
df[Pcol[l:]] = pd.DataFrame(df['P_future'].tolist(), index=df.index)
df[Rcol[:l]] = pd.DataFrame(df['R'].tolist(), index=df.index)
df[Rcol[l:]] = pd.DataFrame(df['R_future'].tolist(), index=df.index)

# 기존 데이터와 t+13 columns drop
df = df.drop(labels="logP_t+"+str(l), axis=1)
df = df.drop(labels="R_t+"+str(l), axis=1)
df = df.drop(labels=['P', 'P_future', 'R', 'R_future'], axis=1)

df.columns

Index(['ticker', 'company', 'timestamp', 'financial stability and liquidity',
       'strong management team', 'competitive advantage', 'market potential',
       'growth prospects', 'diversification within the company',
       'sustainable business model', 'innovation and R&D',
       'corporate governance', 'strong brand recognition', 'open', 'high',
       'low', 'close', 'volume', 'return', 'SPX', 'calculated_price', 'year',
       'month', 'day', 'logP_t-12', 'logP_t-11', 'logP_t-10', 'logP_t-9',
       'logP_t-8', 'logP_t-7', 'logP_t-6', 'logP_t-5', 'logP_t-4', 'logP_t-3',
       'logP_t-2', 'logP_t-1', 'logP_t', 'logP_t+1', 'logP_t+2', 'logP_t+3',
       'logP_t+4', 'logP_t+5', 'logP_t+6', 'logP_t+7', 'logP_t+8', 'logP_t+9',
       'logP_t+10', 'logP_t+11', 'logP_t+12', 'R_t-12', 'R_t-11', 'R_t-10',
       'R_t-9', 'R_t-8', 'R_t-7', 'R_t-6', 'R_t-5', 'R_t-4', 'R_t-3', 'R_t-2',
       'R_t-1', 'R_t', 'R_t+1', 'R_t+2', 'R_t+3', 'R_t+4', 'R_t+5', 'R_t+6',
       'R_t+7', 'R_t+8', '

## log

In [ ]:
import math

# P값들에 대해 log 취하기
for c in [s for s in df.columns if s.find('log')==0]:
  df[c] = df[c].apply(lambda x: math.log(x))

# volume에 대해 log 취하기
df['volume'] = df['volume'].apply(lambda x: math.log(x))

df['volume']

## categorical 변수 처리

In [14]:
# categorical 변수를 int형으로 변경하는 class define

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)

    def fit_transform(self, X: pd.DataFrame, columns: list):
        if not isinstance(columns, list):
            columns = [columns]

        output = X.copy()
        output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].fit_transform(x))

        return output

    def inverse_transform(self, X: pd.DataFrame, columns: list):
        if not isinstance(columns, list):
            columns = [columns]

        if not all(key in self.encoder_dict for key in columns):
            raise KeyError(f"at least one of {columns} is not encoded before")
        output = X.copy()
        try:
            output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].inverse_transform(x))
        except ValueError:
            print(f"Need assignment for 'fit_transform' function")
            raise

        return output

In [15]:
# categorical 변수에는 무엇이 있는지 확인
features = df.columns
cat_cols = list(df.select_dtypes(include=['category', 'object']).columns)
cat_cols

['ticker', 'company']

In [ ]:
# 위에서 만든 MultiColLabelEncoder를 이용해 LabelEncoding 후 확인
mcle = MultiColLabelEncoder()
df_mcle = mcle.fit_transform(df, cat_cols)

# ticker와 company는 같은 내용을 전달하므로 company는 drop한다. 
df_mcle = df_mcle.drop(labels='company', axis=1)

df_mcle['ticker'].unique()

## Train-Validation-Test set split

# timestamp로 sort
df_mcle = df_mcle.sort_values(by=['timestamp', 'ticker'], ascending=True)
df_mcle

In [17]:
# 'timestamp'를 기준으로 2020-09-01, 2021-09-01
train_set = df_mcle[df_mcle['timestamp'] < '2020-09-01']
val_set = df_mcle.loc[(df_mcle['timestamp'] > '2020-09-01') & (df_mcle['timestamp'] < '2021-09-01')]
test_set = df_mcle[df_mcle['timestamp'] > '2021-09-01']

In [18]:
print(len(train_set))
print(len(test_set))
print(len(val_set))

170818
33004
23192


In [19]:
# t+1 ~ t+12 시점이 다음 단계로 넘어가는 문제를 커버하기 위해 data set에 제한을 둠
train_set = train_set.iloc[:-12, :]
val_set = val_set.iloc[:-12, :]
test_set = test_set.iloc[:-12, :]

In [20]:
# train - validation - test set의 분할 비율을 알아보고 그래프 그리기 위해 count 생성

# 날짜 별 count 계산
count = df_mcle.groupby('timestamp').size().reset_index(name='count')

# 누적 합 계산
count['cumsum'] = count['count'].cumsum()

# 누적 비율 계산
count['percent'] = count['cumsum'].apply(lambda x: x/len(df_mcle))
count

,timestamp,count,cumsum,percent
0,2013-05-03,446,446,0.001965
1,2013-05-10,446,892,0.003929
2,2013-05-17,446,1338,0.005894
3,2013-05-24,446,1784,0.007859
4,2013-05-31,446,2230,0.009823
...,...,...,...,...
504,2022-12-30,446,225230,0.992141
505,2023-01-06,446,225676,0.994106
506,2023-01-13,446,226122,0.996071
507,2023-01-20,446,226568,0.998035


In [ ]:

# train - validation - test set의 분할 비율을 알아보고 그래프
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=count.loc[count['timestamp']<'2020-09-01']['timestamp'], y=count.loc[count['timestamp']<'2020-09-01']['percent'], mode = 'lines', name = 'Train Set'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=count.loc[(count['timestamp']>'2020-09-01') & (count['timestamp']<'2021-09-01')]['timestamp'], y=count.loc[(count['timestamp']>'2020-09-01') & (count['timestamp']<'2021-09-01')]['percent'], mode = 'lines', name = 'Validation Set'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=count.loc[count['timestamp']>'2021-09-01']['timestamp'], y=count.loc[count['timestamp']>'2021-09-01']['percent'], mode = 'lines', name = 'Train Set'),
    secondary_y=False,
)

# train: ~ 2020-08-28
# valid: 2020-09-04 ~ 2021-08-27
# test: 2021-09-03 ~

# split의 기준이 되는 점 표시
fig.add_trace(go.Scatter(x=['2020-09-01'], y=[0.75], marker_size = 16, name = 'Train-Validation Split Point'))
fig.add_trace(go.Scatter(x=['2021-08-30'], y=[0.85], marker_size = 16, name = 'Validation-Test Split Point'))

# annotation 추가
fig.add_annotation(x='2017-01-01', y=0.43,
            text="Train Set (75%)",
            showarrow=False,
            font_size=13,
            textangle=-14.5
            )
fig.add_annotation(x='2021-03-01', y=0.88,
            text="Validation Set (10%)",
            showarrow=False,
            font_size=13,
            textangle=-14.5
            )
fig.add_annotation(x='2022-06-01', y=0.89,
            text="Test Set (15%)",
            showarrow=False,
            font_size=13,
            textangle=-14.5
            )
fig.show()

In [22]:
# 'timestamp'는 tabnet이 사용하지 못하므로 drop
train_set = train_set.drop(labels='timestamp', axis=1)
val_set = val_set.drop(labels='timestamp', axis=1)
test_set = test_set.drop(labels='timestamp', axis=1)

In [23]:
train_set.columns

Index(['ticker', 'financial stability and liquidity', 'strong management team',
       'competitive advantage', 'market potential', 'growth prospects',
       'diversification within the company', 'sustainable business model',
       'innovation and R&D', 'corporate governance',
       'strong brand recognition', 'open', 'high', 'low', 'close', 'volume',
       'return', 'SPX', 'calculated_price', 'year', 'month', 'day',
       'logP_t-12', 'logP_t-11', 'logP_t-10', 'logP_t-9', 'logP_t-8',
       'logP_t-7', 'logP_t-6', 'logP_t-5', 'logP_t-4', 'logP_t-3', 'logP_t-2',
       'logP_t-1', 'logP_t', 'logP_t+1', 'logP_t+2', 'logP_t+3', 'logP_t+4',
       'logP_t+5', 'logP_t+6', 'logP_t+7', 'logP_t+8', 'logP_t+9', 'logP_t+10',
       'logP_t+11', 'logP_t+12', 'R_t-12', 'R_t-11', 'R_t-10', 'R_t-9',
       'R_t-8', 'R_t-7', 'R_t-6', 'R_t-5', 'R_t-4', 'R_t-3', 'R_t-2', 'R_t-1',
       'R_t', 'R_t+1', 'R_t+2', 'R_t+3', 'R_t+4', 'R_t+5', 'R_t+6', 'R_t+7',
       'R_t+8', 'R_t+9', 'R_t+10', 'R_t+1

## X, y split

In [24]:
# 사용할 column만 뽑아서 X, y로 나눔
# ticker~GPTscore: 0~10
# SPX~date: 11~21
# P_t-12 ~ P_t: 22~34
# P_t ~ P_t+12: 34~46
# R_t-12 ~ R_t: 47~59
# R_t ~ R_t+12: 59~71

# pX_train = train_set.iloc[:, list(range(36))]
# py_train = train_set.iloc[:, list(range(35, 48))]
# pX_test = test_set.iloc[:, list(range(36))]
# py_test = test_set.iloc[:, list(range(35, 48))]

rX_train = train_set.iloc[:, list(range(22)) + list(range(47, 60))]
ry_train = train_set.iloc[:, list(range(59, 72))]
rX_val = val_set.iloc[:, list(range(22)) + list(range(47, 60))]
ry_val = val_set.iloc[:, list(range(59, 72))]
rX_test = test_set.iloc[:, list(range(22)) + list(range(47, 60))]
ry_test = test_set.iloc[:, list(range(59, 72))]

ry_test.columns

Index(['R_t', 'R_t+1', 'R_t+2', 'R_t+3', 'R_t+4', 'R_t+5', 'R_t+6', 'R_t+7',
       'R_t+8', 'R_t+9', 'R_t+10', 'R_t+11', 'R_t+12'],
      dtype='object')

In [25]:
# 파라미터 다 정하고 돌릴거
final_train = df_mcle[df_mcle['timestamp'] < '2021-09-01']
final_test = df_mcle[df_mcle['timestamp'] > '2021-09-01']

final_train = final_train.iloc[:-12, :]
final_test = final_test.iloc[:-12, :]

final_train = final_train.drop(labels='timestamp', axis=1)
final_test = final_test.drop(labels='timestamp', axis=1)

final_train_X = final_train.iloc[:, list(range(22)) + list(range(47, 60))]
final_train_y = final_train.iloc[:, list(range(59, 72))]
final_test_X = final_test.iloc[:, list(range(22)) + list(range(47, 60))]
final_test_y = final_test.iloc[:, list(range(59, 72))]

In [26]:
len(rX_test)

32992

# Model Running

## Import, Install

In [27]:
import torch

# from pytorch_tabnet.tab_model import TabNetRegressor
# from sklearn.model_selection import KFold

# import optuna
# from optuna.samplers import TPESampler

MAX_EPOCH=300
tabnet_params = dict(n_d=24, n_a=24, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     )

In [ ]:
!pip install pytorch-tabnet

In [ ]:
!pip install torch --upgrade

## Run Model

In [30]:
MAX_EPOCH=300
tabnet_params = dict(n_d=32, n_a=32, n_steps = 3, gamma=1,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     )

In [ ]:
# R, R_future에 대해 model 실행
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor

# TabNetRegressor 모델 생성
model = TabNetRegressor(**tabnet_params)

# 모델 훈련
model.fit(
    X_train=final_train_X.values,
    y_train=final_train_y.values,
    eval_set=[(final_test_X.values, final_test_y.values)],
    eval_name=['val'],
    eval_metric=['rmse'],
    max_epochs=MAX_EPOCH,
    # patience=10,
    # batch_size=256,
    # virtual_batch_size=32,
    # num_workers=0,
    # drop_last=False,
    # from_unsupervised=None,
    # scheduler_params=None,
    # scheduler_fn=None,
    # mask_type='entmax',
    # mask_kwargs=None,
)

# 모델 성능 평가
y_pred = model.predict(final_test_X.values)
rmse = np.sqrt(np.mean(np.square(y_pred - final_test_y.values)))
print(f'RMSE: {rmse}')

In [32]:
import pickle

# 모델 저장
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# 모델 로드
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Graph

## 회사 상관 없이 범위를 입력하면 각 observation을 하나씩 따로 뽑기

In [33]:
import plotly.graph_objs as go

In [60]:
### 함수 완성 1번
def RfromDF(df:pd.DataFrame) -> pd.DataFrame:
    """
    정규화 전 R을 나랑 같은 형태로 불러오기 위해서
    """
    import ast
    R = df.copy()
    R = R[['R', 'R_future']]
    R['R'] = R['R'].apply(ast.literal_eval)
    R['R_future'] = R['R_future'].apply(ast.literal_eval)

    l = 13

    lst = [i for i in range(-l+1, l+1)]
    Rcol= []

    for i in range(2*l):
      if lst[i] < 0:
        col_name = "R_t" + str(lst[i])
        Rcol.append(col_name)
      elif lst[i] == 0:
        col_name = "R_t"
        Rcol.append(col_name)
      else:
        col_name = "R_t+" + str(lst[i])
        Rcol.append(col_name)
    R[Rcol[:l]] = pd.DataFrame(R['R'].tolist(), index=df.index)
    R[Rcol[l:]] = pd.DataFrame(R['R_future'].tolist(), index=df.index)
    R = R.drop(labels = ['R', 'R_future', 'R_t+13'], axis=1)

    return R

In [101]:
### 두번째함수 완성
def RtoP(X_test:pd.DataFrame, y_test_df:pd.DataFrame, y_pred_df:pd.DataFrame, df:pd.DataFrame):
    """
    맨 처음 파일 불러온 직후 dataframe을 df에 넣어주세요! 정규화 전 close, R을 불러오기 위함입니다!
    
    결과:
    dataframe P가 return 되어, 각 observation에 대해 P_t-12, P_t-11, ~ , P_t+12까지의 column에 각각의 P값이 들어 있다. 
    단, index는 초기화된다. 
    """
    # X_test, y_test_df, y_pred_df와 같은 범위의 data를 뽑는다. 
    data = df.copy()
    data = data[data['timestamp'] > '2021-09-01'] ############################## test set 바꾸면 여기도 수정!

    # 계산의 편의성을 위해 'R'에서 R값들을 dataframe 형태로 가져온다. 
    R = RfromDF(data)
    Rpred = y_pred_df.copy()
    # P_t인 close값을 가져온다. 
    P_t = data['close'].values.tolist()

    # 데이터 차원 확인
    # print("X_test", len(X_test))
    # print("y_test", len(y_test_df))
    # print("Rpred", len(Rpred))
    # print("P_t", len(P_t))

    # P columns 만들기
    l = 13
    lst = [i for i in range(-l+1, l)]
    Pcol= []
    for i in range(2*l-1):
      if lst[i] < 0:
        col_name = "P_t" + str(lst[i])
        Pcol.append(col_name)
      elif lst[i] == 0:
        col_name = "P_t"
        Pcol.append(col_name)
      else:
        col_name = "P_t+" + str(lst[i])
        Pcol.append(col_name)
    # dataframe P를 만들기 위해 P_bucket, P_temp를 이용
    P_bucket = []
    P_temp = []

    pred_bucket = []
    pred_temp = []

    for i in range(len(X_test)):
      P_temp.append(P_t[i])
      pred_temp.append(P_t[i])
      for j in range(1, l):
        P_temp.append((R.iloc[i][f'R_t+{j}'] + 1) * P_temp[-1])
        pred_temp.append((Rpred.iloc[i][j] + 1) * pred_temp[-1])
      for j in range(1, l):
        if j == 1: 
          P_temp.insert(-1, P_temp[0] / (R.iloc[i]['R_t'] + 1))      
        else: 
          P_temp.insert(-1, P_temp[0] / (R.iloc[i][f'R_t-{j}'] + 1))      
      P_bucket.append(P_temp)
      pred_bucket.append(pred_temp)
      P_temp = []
      pred_temp = []
      if i % 10000 == 0:
        print(" real i ", i)


    P = pd.DataFrame(P_bucket, columns=Pcol)
    P_pred = pd.DataFrame(pred_bucket, columns=Pcol[l-1:])
    X_test = pd.concat([X_test.reset_index(drop=True), P.iloc[:, :l]], axis=1)
    y_test_df = pd.concat([y_test_df.reset_index(drop=True), P.iloc[:, l:]], axis=1)

    y_pred_df = pd.concat([y_pred_df.reset_index(drop=True), P_pred], axis=1)
    return X_test, y_test_df, y_pred_df

In [77]:
!pip install -U kaleido
import plotly.io as pio
pio.renderers.default = 'colab'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
### 세번째함수
from plotly.subplots import make_subplots
import os

def drawGraphCompanyy(X_test:pd.DataFrame, y_test_df:pd.DataFrame, y_pred:np.ndarray, df: pd.DataFrame): 
  # 데이터 준비
  # X_test로부터 ticker, company 정보 꺼내기
  mcle_ticker_df = pd.DataFrame(X_test.copy().iloc[:, 0])
  mcle_ticker_lst = mcle_ticker_df.values.tolist()
  TICKER = mcle.inverse_transform(X_test, 'ticker')
  TICKER = TICKER['ticker'].values.tolist()

  # X_test로부터 'timestamp' column 다시 만들기
  test_timestamp = X_test.loc[:, ['year', 'month', 'day']]
  test_timestamp['timestamp'] = test_timestamp['year'].astype(str) + '-' + test_timestamp['month'].astype(str) + '-' + test_timestamp['day'].astype(str)
  test_timestamp['timestamp'] = pd.to_datetime(test_timestamp['timestamp'])

  # X_test, y_test, y_pred를 그래프 뽑기 편하게 만들어주기 (timestamp 붙이기, ticker decode)
  X_test_graph = X_test.copy()
  X_columns = list(X_test_graph.columns)
  X_test_graph = pd.concat([X_test_graph, test_timestamp['timestamp']], ignore_index=True, axis=1)
  X_test_graph.columns = X_columns + ['timestamp']

  y_test_graph = y_test_df.copy()
  y_columns = list(y_test_graph.columns)
  y_test_graph = pd.concat([y_test_graph, test_timestamp['timestamp']], ignore_index = True, axis=1)
  y_test_graph = pd.concat([y_test_graph, X_test_graph['ticker']], ignore_index = True, axis=1)
  y_test_graph.columns = y_columns+ ['timestamp', 'ticker']

  y_pred_graph = pd.DataFrame(y_pred)
  y_pred_graph = pd.concat([y_pred_graph, test_timestamp['timestamp'].reset_index(drop=True)], ignore_index = True, axis = 1)
  y_pred_graph = pd.concat([y_pred_graph, X_test_graph['ticker'].reset_index(drop=True)], ignore_index=True, axis=1)
  y_pred_graph.columns = [f"P{s}" for s in range(13)] + ['timestamp', 'ticker']

  # # timelist (X of graph)
  X_timelist = X_test_graph['timestamp'].unique().tolist()
  X_timelist.sort()
  y_timelist = y_test_graph['timestamp'].unique().tolist()
  y_timelist.sort()

  X_test_graph, y_test_graph, y_pred_graph = RtoP(X_test_graph, y_test_graph, y_pred_graph, df)

  # 그래프 그리기
  unique_TICKER = list(set(TICKER))
  for t in unique_TICKER:
    coIndex = TICKER.index(t)
    co_n = mcle_ticker_lst[coIndex]
    
    # 특정 ticker 골라내기
    X_temp = X_test_graph[X_test_graph['ticker'] == co_n[0]]
    y_temp = y_test_graph[y_test_graph['ticker'] == co_n[0]]
    pred_temp = y_pred_graph[y_pred_graph['ticker'] == co_n[0]]

    # sort
    X_temp = X_temp.sort_values(by='timestamp', ascending=True)
    y_temp = y_temp.sort_values(by='timestamp', ascending=True)
    pred_temp = pred_temp.sort_values(by='timestamp', ascending=True)

    # 그래프를 그릴 P만 골라내기
    X_graph = X_temp.iloc[:, -l:]
    y_graph = y_temp.iloc[:, -l:]
    pred_graph = pred_temp.iloc[:, -l:]

    # X dataset 마련
    Xlist = [0] * len(X_timelist)
    for i in range(len(X_graph)-l): 
      for j in range(l):
        Xlist[i+j] += X_graph.iloc[i, j]
    Xlist_real = []
    for i in range(len(Xlist)):
      if i<len(Xlist)/2:
        Xlist_real.append(Xlist[i]/i)
      else:
        Xlist_real.append(Xlist[i]/(len(Xlist)-i))

    # y dataset 마련
    ylist = [0] * len(y_timelist)
    for i in range(len(y_graph)-l): 
      for j in range(l):
        ylist[i+j] += y_graph.iloc[i, j]
    ylist_real = []
    for i in range(len(ylist)):
      if i<len(ylist)/2:
        ylist_real.append(ylist[i]/i)
      else:
        ylist_real.append(ylist[i]/(len(ylist)-i))

    # pred dataset 마련
    predlist = [0] * len(y_timelist)
    for i in range(len(pred_graph)-l): 
      for j in range(l):
        predlist[i+j] += pred_graph.iloc[i, j]
    predlist_real = []
    for i in range(len(predlist)):
      if i<len(predlist)/2:
        predlist_real.append(predlist[i]/i)
      else:
        predlist_real.append(predlist[i]/(len(predlist)-i))

    # 그래프 그리기
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(title_text=t, title_font_size=20)

    Xrange = [i for i in range(len(X_timelist))]
    yrange = [i + len(X_timelist) - 2 for i in range(len(y_timelist))]

    # Add traces
    fig.add_trace(
        go.Scatter(x=Xrange, y=Xlist_real, name="X_test"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=yrange, y=ylist_real, name="y_test"),
        secondary_y=False,
        #secondary_y=True,
    )

    fig.add_trace(
        go.Scatter(x=yrange, y=predlist_real, name="y_pred"),
        secondary_y=False,
        #secondary_y=True,
    )
    if not os.path.exists('graph'):
      os.makedirs('graph')

    from io import BytesIO

    import matplotlib.pyplot as plt
    img_bytes = fig.to_image(format="png", engine="kaleido")
    img_arr = plt.imread(BytesIO(img_bytes))
    plt.imsave(f'graph/picture{t}.png', img_arr)


    # fig.show()
    # break;

In [ ]:
drawGraphCompanyy(final_test_X, final_test_y, y_pred, df_)